# Probability Amplitude Swapping Implementation 
### Parallelized and Parameterized

Date: Dec 20, 2023

Algorithm Creator: Iain Burge

Code Authors: Iain Burge (Used some of Tom Mai's Code as reference)

Paper Authors: Iain Burge, Tom Mai, Michel Barbeau

### Import Modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import MCXGate
from qiskit.quantum_info import Statevector

### Define Variables

In [2]:
x = 10
y = 13
num_bits = 4

if num_bits is None:
    num_bits = int(np.ceil(np.log2(max(x, y))))
m = int(np.ceil(np.log2(num_bits)))


### Define $U_{i,j}$, $a_{i,j}$, $b_{i,j}$, and $c_i$ functions

In [3]:
def bitsToString(value: int, nbits: int=None):
    if nbits is None:
        nbits = int(np.ceil(np.log2(value)))

    return f"{value:0>{nbits}b}"

def extractBit(bit: int, basis_state: int, 
        nbits: int = num_bits
    ) -> int:
    """Leftmost bit is 0"""
    return (basis_state & (1<<(nbits-bit-1))) >> (nbits-bit-1)

def setBit(bit: int, value: int, basis_state: int, 
        nbits: int = num_bits
    ) -> int:
    """Leftmost bit is 0"""
    basis_state &= ~(1<<(nbits-bit-1))
    basis_state ^= value * (1<<(nbits-bit-1))

    return basis_state

def calculateAB(i: int, j: int, x_i: int = x, y_i: int = y,
        nbits: int = num_bits
    ) -> tuple[bool, bool]:
    top_bit     = (2**i) * (2*j)
    bottom_bit  = (2**i) * (2*j + 1)

    if bottom_bit >= num_bits:
        return False, False
    
    a_ij =\
        extractBit(top_bit, x_i, nbits)    == extractBit(top_bit, y_i, nbits)
    b_ij =\
        extractBit(bottom_bit, x_i, nbits) != extractBit(bottom_bit, y_i, nbits)

    return a_ij, b_ij

def calculateC(i: int, x_m: int, nbits: int = num_bits) -> bool:
    return extractBit(i, x_m, nbits) == 0 

def U(i: int, j: int, a_ij: bool, b_ij: bool, basis_state: int, 
        nbits: int = num_bits,
    ) -> int:
    top_bit    = (2**i) * (2*j)
    bottom_bit = (2**i) * (2*j + 1)

    if bottom_bit >= num_bits:
        return basis_state

    top_value    = extractBit(top_bit, basis_state, nbits=num_bits)
    bottom_value = extractBit(bottom_bit, basis_state, nbits=num_bits)

    if a_ij and b_ij:
        top_value ^= bottom_value
    if b_ij:
        bottom_value ^= top_value
    
    basis_state = setBit(top_bit, top_value, basis_state, nbits)
    basis_state = setBit(bottom_bit, bottom_value, basis_state, nbits)

    return basis_state

# test = 14
# print(bitsToString(test, 8))
# test = setBit(4, 0, 14, 8)
# print(bitsToString(test, 8))
# print(extractBit(0,test,8))


### Calculate $a_{i,j}$, $b_{i,j}$, and $c_i$

In [4]:
x_i = x
y_i = y
a = []
b = []
c = []

for i in range(m):
    a.append([])
    b.append([])

    for j in range(2**(m-i-1)):
        a_ij, b_ij = calculateAB(i, j, x_i, y_i, num_bits)
        a[i].append(a_ij)
        b[i].append(b_ij)

        x_i = U(i, j, a_ij, b_ij, x_i, num_bits)
        y_i = U(i, j, a_ij, b_ij, y_i, num_bits)

c.append(False)
for i in range(1, num_bits):
    c.append(calculateC(i, x_m=x_i, nbits=num_bits))
        

print(f"{a=}")
print(f"{b=}")
print(f"{c=}")


a=[[True, False, False, False], [True, False], [False]]
b=[[False, True, False, False], [True, False], [True]]
c=[False, False, True, False, True]


### Build the Circuit

In [5]:
circuit = QuantumCircuit(num_bits)

#Step 1:
for i in range(num_bits):
    for j in range(2**(num_bits-i-1)):
        top_bit     = (2**i) * (2*j)
        bottom_bit  = (2**i) * (2*j + 1)

        if bottom_bit >= num_bits:
            break

        if a[i][j] and b[i][j]:
            circuit.cx(bottom_bit, top_bit)
        if b[i][j]:
            circuit.cx(top_bit, bottom_bit)

    circuit.barrier()

#Step 2:
for k in range(num_bits):
    if c[k]:
        circuit.x(k)

circuit.append(
    MCXGate(num_bits-1), 
    [k for k in range(1, 2**m)]+[0]
)

for k in range(num_bits):
    if c[k]:
        circuit.x(k)

#Step 3:
for i in range(num_bits-1, -1, -1):
    circuit.barrier()
    for j in range(2**(num_bits-i-1)):
        top_bit     = (2**i) * (2*j)
        bottom_bit  = (2**i) * (2*j + 1)

        if bottom_bit >= num_bits:
            break

        if b[i][j]:
            circuit.cx(top_bit, bottom_bit)
        if a[i][j] and b[i][j]:
            circuit.cx(bottom_bit, top_bit)


circuit.draw(output="mpl")

CircuitError: 'Index 5 out of range for size 5.'

### Testing

In [ ]:
for z in range(2**num_bits):
    outputText = ''
    bitString = bitsToString(z, num_bits)[::-1]

    state = Statevector.from_label(bitString)

    state = state.evolve(circuit)

    result = state.draw('latex_source')
    result = ''.join(i for i in result if i.isdigit())
    resultInt = int(f"0b{result[::-1]}", base=2)

    outputText += f"init: |{z:>02}> = {state.draw('latex_source')}"
    outputText += f"\t||\tFinal: |{resultInt:>02}> = {state.draw('latex_source')}\t"
    outputText += "Id" if z == resultInt else "Transposed"

    #print results
    print(outputText.replace(r'\rangle', '>') + "\n")